[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tulasiram58827/craft_tflite/blob/main/colabs/savedmodel_to_tflite.ipynb)

This notebook converts tensorflow savel model to tflite version

## SetUp

In [ ]:
"""
Authors
 * Tulasi Ram
"""
!pip install tf-nightly
!pip install onnx
!pip install git+https://github.com/onnx/onnx-tensorflow.git

import onnx

import tensorflow as tf
from onnx_tf.backend import prepare

### Export to Tensorflow Graph

In [ ]:
# Export model to tensorflow

!wget https://github.com/tulasiram58827/craft_tflite/blob/main/models/craft.onnx?raw=true -O craft.onnx
onnx_model = onnx.load('/content/craft.onnx')

tf_rep = prepare(onnx_model)
tf_rep.export_graph('craft.pb')

print("Model converted to tensorflow graph succesfully.")

In [14]:
!gdown https://drive.google.com/uc?id=1HvujMMNCG49esPpZOHEnPFY6k-khbrmZ
!unzip /content/image_files.zip
dataset_path = '/content/image_files/'


loaded = tf.saved_model.load('/content/craft.pb')

concrete_func = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

concrete_func.inputs[0].set_shape([None, 3, 320, 200])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

converter.optimizations = [tf.lite.Optimize.DEFAULT]

Downloading...
From: https://drive.google.com/uc?id=1HvujMMNCG49esPpZOHEnPFY6k-khbrmZ
To: /content/image_files.zip
7.30MB [00:00, 64.1MB/s]
Archive:  /content/image_files.zip
replace image_files/000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: image_files/000.jpg     
  inflating: image_files/001.jpg     
  inflating: image_files/002.jpg     
  inflating: image_files/003.jpg     
  inflating: image_files/004.jpg     
  inflating: image_files/005.jpg     
  inflating: image_files/006.jpg     
  inflating: image_files/007.jpg     
  inflating: image_files/008.jpg     
  inflating: image_files/009.jpg     
  inflating: image_files/010.jpg     
  inflating: image_files/011.jpg     
  inflating: image_files/012.jpg     
  inflating: image_files/013.jpg     
  inflating: image_files/014.jpg     
  inflating: image_files/015.jpg     
  inflating: image_files/016.jpg     
  inflating: image_files/017.jpg     
  inflating: image_files/018.jpg     
  inflating: image_files/019.jpg  

In [15]:
# Uncomment this line for float16 quantization.
# converter.target_spec.supported_types = [tf.float16]
# converter.target_spec.supported_types = [tf.lite.OpsSet.TFLITE_BUILTINS]


# Uncomment For Integer Quantization
# def representative_data_gen():
#     for file in os.listdir(dataset_path)[:10]:
#         file_path = dataset_path+file
#         image = imgproc.loadImage(file_path)
#         image = cv2.resize(image, dsize=(800, 1280), interpolation=cv2.INTER_LINEAR)
#         img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
#         ratio_h = ratio_w = 1 / target_ratio

#         # preprocessing
#         x = imgproc.normalizeMeanVariance(img_resized)
#         x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
#         x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
#         x = x.cpu().detach().numpy()
#         yield [x]

# converter.representative_dataset = representative_data_gen

tf_lite_model = converter.convert()

INFO:absl:Using new converter: If you encounter a problem please file a bug. You can opt-out by setting experimental_new_converter=False


ConverterError: ignored

In [ ]:
open('craft.tflite', 'wb').write(tf_lite_model)

print("Converted to tensorflow lite succesfully.")